In [2]:
import cv2
import numpy as np

# Load the HSV reference image
hsv_reference = cv2.imread("hsv-0.png")
if hsv_reference is None:
    print("Warning: The file 'hsv-0.png' was not found. HSV reference image will not be displayed.")
    hsv_reference = np.zeros((100, 300, 3), dtype=np.uint8)  # Create a blank image as a placeholder

def detect_fire(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Adjusted HSV range for fire-like colors (red, orange, yellow)
    lower_bound1 = np.array([0, 50, 50], dtype=np.uint8)   # Red hues
    upper_bound1 = np.array([10, 255, 255], dtype=np.uint8)
    lower_bound2 = np.array([160, 50, 50], dtype=np.uint8) # Red hues (wrap-around)
    upper_bound2 = np.array([179, 255, 255], dtype=np.uint8)
    lower_bound3 = np.array([18, 100, 100], dtype=np.uint8) # Orange-yellow hues
    upper_bound3 = np.array([35, 255, 255], dtype=np.uint8)
    
    # Combine masks for red and orange-yellow hues
    mask1 = cv2.inRange(hsv, lower_bound1, upper_bound1)
    mask2 = cv2.inRange(hsv, lower_bound2, upper_bound2)
    mask3 = cv2.inRange(hsv, lower_bound3, upper_bound3)
    mask = cv2.bitwise_or(mask1, mask2)
    mask = cv2.bitwise_or(mask, mask3)
    
    # Morphological operations to clean up the mask
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    # Find contours and filter based on size and shape
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        if cv2.contourArea(contour) > 1000:  # Increased area threshold for better accuracy
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = float(w) / h
            if 0.5 < aspect_ratio < 2.0:  # Filter based on aspect ratio to avoid false positives
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
                cv2.putText(frame, "Fire Detected!", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
    return frame, mask

cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break
    processed_frame, mask = detect_fire(frame)
    
    # Resize HSV reference image
    hsv_ref_resized = cv2.resize(hsv_reference, (300, 100))
    
    # Display results
    cv2.imshow("Fire Detection", processed_frame)
    cv2.imshow("Mask", mask)
    cv2.imshow("HSV Reference", hsv_ref_resized)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 